[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/kata_05_temple_visitor_spiritual_observer_unrevised.ipynb)

## 🏮 The Ancient Scroll Unfurls 🏮

# THE MASTER'S EYE: CATEGORIZING THE SPIRITUAL JOURNEY
Dan Level: 1 (Temple Sweeper) | Time: 45 minutes | Sacred Arts: Multi-variable Features, Multi-class Classification, Softmax Activation

## 📜 THE CHALLENGE

Master Pai-Torch sits in silent observation at the temple entrance, watching the endless stream of visitors arrive for guidance. Through decades of patient study, the ancient master has learned to categorize each seeker into one of three sacred stages: **nervous novices** who fidget and rush with endless questions, **steady practitioners** who move with growing confidence and measured inquiry, and **serene masters** who glide with graceful slowness, breathing deeply, rarely needing to ask what they already know within.

"Grasshopper," Master Pai-Torch says, eyes still fixed on a visitor approaching the gates, "the untrained eye sees only people walking. But observe closely—the way one moves reveals the state of one's inner cultivation. Their walking speed speaks of anxiety or calm, their posture shows discipline or distraction, their breathing patterns reveal tension or peace, and their questioning frequency exposes wisdom or confusion. Today, you shall learn to see as the masters see, using the mystical arts of multi-variable classification to peer into the soul through observable behavior."

## 🎯 THE SACRED OBJECTIVES

- [ ] Master the creation of synthetic data with multiple meaningful features
- [ ] Build a neural network that processes multiple input variables simultaneously
- [ ] Implement multi-class classification using softmax activation
- [ ] Understand how different features contribute to classification decisions
- [ ] Visualize the relationship between multiple behavioral patterns and spiritual development
- [ ] Learn to interpret model confidence across multiple classes

In [ ]:
# 📦 FIRST CELL - ALL IMPORTS AND CONFIGURATION
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple
import seaborn as sns

# Set reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Global configuration constants
DEFAULT_CHAOS_LEVEL = 0.15
SACRED_SEED = 42
N_FEATURES = 4  # walking_speed, posture_score, question_frequency, breathing_depth
N_CLASSES = 3   # nervous_novice=0, steady_practitioner=1, serene_master=2

In [ ]:
# 🧘 THE SACRED DATA GENERATION SCROLL

def generate_temple_visitor_data(n_visitors: int = 300, chaos_level: float = 0.15,
                               sacred_seed: int = 42) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate observations of temple visitors and their spiritual development stage.
    
    Master Pai-Torch's ancient wisdom reveals these patterns:
    - Nervous Novices: Fast walking (0.8-1.2), poor posture (0.2-0.5), many questions (8-15), shallow breathing (0.3-0.6)
    - Steady Practitioners: Moderate walking (0.5-0.8), good posture (0.6-0.8), some questions (3-8), regular breathing (0.6-0.8)
    - Serene Masters: Slow walking (0.2-0.5), excellent posture (0.8-1.0), few questions (0-3), deep breathing (0.8-1.0)
    
    Args:
        n_visitors: Number of temple visitors to observe
        chaos_level: Amount of individual variation (0.0 = perfectly predictable, 1.0 = pure chaos)
        sacred_seed: Ensures consistent observations
        
    Returns:
        Tuple of (behavioral_features, spiritual_stage_labels) as tensors
        Features: [walking_speed, posture_score, question_frequency, breathing_depth]
        Labels: 0=Nervous Novice, 1=Steady Practitioner, 2=Serene Master
    """
    torch.manual_seed(sacred_seed)
    
    # Create equal representation of each spiritual stage
    visitors_per_stage = n_visitors // N_CLASSES
    remainder = n_visitors % N_CLASSES
    
    features_list = []
    labels_list = []
    
    # Generate Nervous Novices (label = 0)
    n_novices = visitors_per_stage + (1 if remainder > 0 else 0)
    walking_speed = torch.uniform(0.8, 1.2, (n_novices,))
    posture_score = torch.uniform(0.2, 0.5, (n_novices,))
    question_freq = torch.uniform(8.0, 15.0, (n_novices,))
    breathing_depth = torch.uniform(0.3, 0.6, (n_novices,))
    
    novice_features = torch.stack([walking_speed, posture_score, question_freq, breathing_depth], dim=1)
    novice_labels = torch.zeros(n_novices, dtype=torch.long)
    
    features_list.append(novice_features)
    labels_list.append(novice_labels)
    
    # Generate Steady Practitioners (label = 1)
    n_practitioners = visitors_per_stage + (1 if remainder > 1 else 0)
    walking_speed = torch.uniform(0.5, 0.8, (n_practitioners,))
    posture_score = torch.uniform(0.6, 0.8, (n_practitioners,))
    question_freq = torch.uniform(3.0, 8.0, (n_practitioners,))
    breathing_depth = torch.uniform(0.6, 0.8, (n_practitioners,))
    
    practitioner_features = torch.stack([walking_speed, posture_score, question_freq, breathing_depth], dim=1)
    practitioner_labels = torch.ones(n_practitioners, dtype=torch.long)
    
    features_list.append(practitioner_features)
    labels_list.append(practitioner_labels)
    
    # Generate Serene Masters (label = 2)
    n_masters = visitors_per_stage
    walking_speed = torch.uniform(0.2, 0.5, (n_masters,))
    posture_score = torch.uniform(0.8, 1.0, (n_masters,))
    question_freq = torch.uniform(0.0, 3.0, (n_masters,))
    breathing_depth = torch.uniform(0.8, 1.0, (n_masters,))
    
    master_features = torch.stack([walking_speed, posture_score, question_freq, breathing_depth], dim=1)
    master_labels = torch.full((n_masters,), 2, dtype=torch.long)
    
    features_list.append(master_features)
    labels_list.append(master_labels)
    
    # Combine all data
    all_features = torch.cat(features_list, dim=0)
    all_labels = torch.cat(labels_list, dim=0)
    
    # Add natural variation (individual differences)
    noise = torch.randn_like(all_features) * chaos_level * all_features.std(dim=0)
    all_features = all_features + noise
    
    # Ensure features stay within reasonable bounds
    all_features = torch.clamp(all_features, min=0.0)
    all_features[:, 2] = torch.clamp(all_features[:, 2], max=20.0)  # Max 20 questions
    all_features[:, [0, 1, 3]] = torch.clamp(all_features[:, [0, 1, 3]], max=1.5)  # Reasonable maximums
    
    # Shuffle the data
    shuffle_indices = torch.randperm(n_visitors)
    all_features = all_features[shuffle_indices]
    all_labels = all_labels[shuffle_indices]
    
    return all_features, all_labels

def visualize_spiritual_wisdom(features: torch.Tensor, labels: torch.Tensor, 
                             predictions: torch.Tensor = None, title_suffix: str = ""):
    """
    Display the sacred patterns of spiritual development through Master Pai-Torch's eyes.
    """
    feature_names = ['Walking Speed', 'Posture Score', 'Questions/Hour', 'Breathing Depth']
    stage_names = ['Nervous Novice', 'Steady Practitioner', 'Serene Master']
    colors = ['red', 'orange', 'purple']
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'Master Pai-Torch\'s Spiritual Observation {title_suffix}', fontsize=16, fontweight='bold')
    
    # Plot feature distributions for each class
    for i, feature_name in enumerate(feature_names):
        ax = axes[i//2, i%2] if i < 4 else None
        if ax is None:
            continue
            
        for stage_idx, (stage_name, color) in enumerate(zip(stage_names, colors)):
            stage_mask = labels == stage_idx
            stage_data = features[stage_mask, i].numpy()
            ax.hist(stage_data, alpha=0.6, color=color, label=stage_name, bins=15)
        
        ax.set_xlabel(feature_name)
        ax.set_ylabel('Number of Visitors')
        ax.set_title(f'Distribution: {feature_name}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # Feature correlation heatmap
    ax = axes[1, 2]
    correlation_matrix = torch.corrcoef(features.T).numpy()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                xticklabels=feature_names, yticklabels=feature_names, ax=ax)
    ax.set_title('Feature Correlations\n(Master\'s Insights)')
    
    # 3D scatter plot of selected features
    ax = fig.add_subplot(2, 3, 5, projection='3d')
    for stage_idx, (stage_name, color) in enumerate(zip(stage_names, colors)):
        stage_mask = labels == stage_idx
        stage_features = features[stage_mask]
        ax.scatter(stage_features[:, 0], stage_features[:, 1], stage_features[:, 3],
                  c=color, label=stage_name, alpha=0.6, s=30)
    
    ax.set_xlabel('Walking Speed')
    ax.set_ylabel('Posture Score')
    ax.set_zlabel('Breathing Depth')
    ax.set_title('3D Spiritual Landscape')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # If predictions are provided, show confusion matrix
    if predictions is not None:
        from sklearn.metrics import confusion_matrix, classification_report
        
        # Convert predictions to class labels
        pred_labels = torch.argmax(predictions, dim=1)
        
        # Create confusion matrix
        cm = confusion_matrix(labels.numpy(), pred_labels.numpy())
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=stage_names, yticklabels=stage_names)
        plt.title('Confusion Matrix: Master Pai-Torch\'s Accuracy')
        plt.xlabel('Predicted Spiritual Stage')
        plt.ylabel('True Spiritual Stage')
        plt.show()
        
        # Print classification report
        print("\n🧘 MASTER PAI-TORCH'S DETAILED ASSESSMENT:")
        print(classification_report(labels.numpy(), pred_labels.numpy(), 
                                  target_names=stage_names, digits=3))

# Generate and visualize the sacred data
print("🏮 Master Pai-Torch observes the temple visitors...")
features, labels = generate_temple_visitor_data(n_visitors=300, chaos_level=0.15)

print(f"\n📊 OBSERVATIONS RECORDED:")
print(f"- Total visitors observed: {len(features)}")
print(f"- Features per visitor: {features.shape[1]}")
print(f"- Nervous Novices: {(labels == 0).sum().item()}")
print(f"- Steady Practitioners: {(labels == 1).sum().item()}")
print(f"- Serene Masters: {(labels == 2).sum().item()}")

visualize_spiritual_wisdom(features, labels, title_suffix="(Original Data)")

## 💫 THE SPIRITUAL CLASSIFIER

Now you must build the mystical artifact that can peer into a visitor's soul through their behavior patterns. Master Pai-Torch whispers ancient wisdom about multi-class classification...

In [ ]:
# 🔮 THE SPIRITUAL OBSERVATION NETWORK

class SpiritualObserver(nn.Module):
    """A mystical network that categorizes temple visitors by their spiritual development."""
    
    def __init__(self, input_features: int = 4, hidden_size: int = 8, num_classes: int = 3):
        super(SpiritualObserver, self).__init__()
        
        # TODO: Create the neural architecture for spiritual observation
        # Hint: You'll need layers to process multiple features and output class probabilities
        
        # First hidden layer: transforms behavioral features into hidden wisdom
        self.hidden = None  # TODO: nn.Linear(input_features, hidden_size)
        
        # Output layer: transforms hidden wisdom into spiritual stage probabilities
        self.classifier = None  # TODO: nn.Linear(hidden_size, num_classes)
        
        # Activation functions
        self.relu = nn.ReLU()  # For hidden layer non-linearity
        # NOTE: We don't need to add softmax here - CrossEntropyLoss includes it!
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """
        Channel behavioral observations through the network of spiritual wisdom.
        
        Args:
            features: Tensor of shape (batch_size, 4) containing behavioral measurements
        
        Returns:
            raw_scores: Tensor of shape (batch_size, 3) with raw class scores (logits)
        """
        # TODO: Forward pass through the network
        # Hint: features -> hidden layer -> ReLU -> output layer
        
        # Process through hidden layer with non-linear activation
        hidden_wisdom = None  # TODO: Apply hidden layer and ReLU activation
        
        # Generate final class scores (logits)
        raw_scores = None  # TODO: Apply classifier layer
        
        return raw_scores

def train_spiritual_observer(model: nn.Module, features: torch.Tensor, labels: torch.Tensor,
                           epochs: int = 1000, learning_rate: float = 0.01) -> tuple:
    """
    Train the spiritual observer to recognize different stages of development.
    
    Returns:
        Tuple of (loss_history, accuracy_history)
    """
    # TODO: Choose the appropriate loss function for multi-class classification
    # Hint: CrossEntropyLoss is perfect for this - it combines softmax and negative log likelihood
    criterion = None
    
    # TODO: Choose your optimization method
    # Hint: Adam works well for multi-class problems
    optimizer = None
    
    loss_history = []
    accuracy_history = []
    
    for epoch in range(epochs):
        # TODO: CRITICAL - Clear the gradient spirits from the previous cycle
        
        # TODO: Forward pass - get class score predictions
        raw_scores = None
        
        # TODO: Compute the loss
        loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update parameters
        
        # Calculate accuracy for this epoch
        with torch.no_grad():
            predicted_classes = torch.argmax(raw_scores, dim=1)
            accuracy = (predicted_classes == labels).float().mean().item()
        
        loss_history.append(loss.item())
        accuracy_history.append(accuracy)
        
        # Report progress to Master Pai-Torch
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')
            if accuracy > 0.8:
                print("🌸 Master Pai-Torch observes: 'Your perception grows clearer, Grasshopper.'")
    
    return loss_history, accuracy_history

def get_class_probabilities(model: nn.Module, features: torch.Tensor) -> torch.Tensor:
    """
    Get the probability distribution over classes for each visitor.
    
    Returns:
        Tensor of shape (batch_size, 3) with probabilities summing to 1 for each row
    """
    model.eval()
    with torch.no_grad():
        raw_scores = model(features)
        # TODO: Apply softmax to convert raw scores to probabilities
        # Hint: torch.softmax(raw_scores, dim=1)
        probabilities = None
    return probabilities

# Create and display the spiritual observer
model = SpiritualObserver(input_features=4, hidden_size=8, num_classes=3)
print("🔮 SPIRITUAL OBSERVER ARCHITECTURE:")
print(model)
print(f"\nTotal parameters to learn: {sum(p.numel() for p in model.parameters())}")

## ⚗️ THE TRAINING RITUAL

Time to train your spiritual observer! Watch as it learns to distinguish between the three sacred stages of development.

In [ ]:
# 🏋️ TRAINING THE MYSTICAL PERCEPTION

print("🧘 Master Pai-Torch begins the training ritual...")

# Train the model
loss_history, accuracy_history = train_spiritual_observer(
    model, features, labels, epochs=1000, learning_rate=0.01
)

# Visualize training progress
plt.figure(figsize=(15, 5))

# Loss plot
plt.subplot(1, 3, 1)
plt.plot(loss_history, color='red', alpha=0.7)
plt.title('Loss During Training\n(Spiritual Confusion Decreasing)')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.grid(True, alpha=0.3)

# Accuracy plot
plt.subplot(1, 3, 2)
plt.plot(accuracy_history, color='green', alpha=0.7)
plt.title('Accuracy During Training\n(Spiritual Insight Growing)')
plt.xlabel('Epoch')
plt.ylabel('Classification Accuracy')
plt.grid(True, alpha=0.3)

# Final accuracy distribution
plt.subplot(1, 3, 3)
final_predictions = get_class_probabilities(model, features)
predicted_labels = torch.argmax(final_predictions, dim=1)
stage_names = ['Nervous\nNovice', 'Steady\nPractitioner', 'Serene\nMaster']

# Count correct predictions per class
correct_per_class = []
total_per_class = []
for class_idx in range(3):
    class_mask = labels == class_idx
    correct_mask = (predicted_labels == labels) & class_mask
    correct_per_class.append(correct_mask.sum().item())
    total_per_class.append(class_mask.sum().item())

accuracies = [c/t for c, t in zip(correct_per_class, total_per_class)]
colors = ['red', 'orange', 'purple']

bars = plt.bar(stage_names, accuracies, color=colors, alpha=0.7)
plt.title('Per-Class Accuracy\n(Master\'s Assessment)')
plt.ylabel('Classification Accuracy')
plt.ylim(0, 1)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n🎯 FINAL TRAINING RESULTS:")
print(f"- Final Loss: {loss_history[-1]:.4f}")
print(f"- Final Accuracy: {accuracy_history[-1]:.4f}")
print(f"- Best Accuracy Achieved: {max(accuracy_history):.4f}")

## 🔍 ANALYZING THE MASTER'S VISION

Let's examine how well your spiritual observer has learned to categorize temple visitors.

In [ ]:
# 🎭 DETAILED ANALYSIS OF SPIRITUAL PERCEPTION

# Get final predictions
final_predictions = get_class_probabilities(model, features)
predicted_labels = torch.argmax(final_predictions, dim=1)

# Show comprehensive results with confusion matrix
visualize_spiritual_wisdom(features, labels, final_predictions, "(Trained Model)")

# Analyze some specific examples
print("\n🔮 MASTER PAI-TORCH'S SPECIFIC OBSERVATIONS:")
print("\nExamining individual visitors...")

stage_names = ['Nervous Novice', 'Steady Practitioner', 'Serene Master']
feature_names = ['Walking Speed', 'Posture Score', 'Questions/Hour', 'Breathing Depth']

# Show 5 examples from each class
for true_class in range(3):
    class_mask = labels == true_class
    class_indices = torch.where(class_mask)[0][:5]  # First 5 examples
    
    print(f"\n--- {stage_names[true_class].upper()} EXAMPLES ---")
    
    for idx in class_indices:
        true_label = labels[idx].item()
        pred_label = predicted_labels[idx].item()
        pred_probs = final_predictions[idx]
        visitor_features = features[idx]
        
        print(f"\nVisitor {idx.item()}:")
        print(f"  Behaviors: {[f'{name}: {val:.3f}' for name, val in zip(feature_names, visitor_features)]}")
        print(f"  True stage: {stage_names[true_label]}")
        print(f"  Predicted: {stage_names[pred_label]} {'✓' if pred_label == true_label else '✗'}")
        print(f"  Confidence: {pred_probs[pred_label]:.3f}")
        print(f"  All probabilities: {[f'{name}: {prob:.3f}' for name, prob in zip(stage_names, pred_probs)]}")

# Feature importance analysis
print("\n\n🧠 UNDERSTANDING WHAT THE MODEL LEARNED:")
print("\nHidden layer weights (how features combine):")
hidden_weights = model.hidden.weight.data
print(f"Shape: {hidden_weights.shape} (8 hidden neurons, 4 input features)")

# Show which features each hidden neuron focuses on
for neuron_idx in range(hidden_weights.shape[0]):
    neuron_weights = hidden_weights[neuron_idx]
    dominant_feature = torch.argmax(torch.abs(neuron_weights)).item()
    print(f"Hidden neuron {neuron_idx}: Most sensitive to {feature_names[dominant_feature]} (weight: {neuron_weights[dominant_feature]:.3f})")

print("\nOutput layer weights (how hidden neurons vote for classes):")
output_weights = model.classifier.weight.data
print(f"Shape: {output_weights.shape} (3 classes, 8 hidden neurons)")

for class_idx, class_name in enumerate(stage_names):
    class_weights = output_weights[class_idx]
    strongest_neuron = torch.argmax(torch.abs(class_weights)).item()
    print(f"{class_name}: Strongest connection to hidden neuron {strongest_neuron} (weight: {class_weights[strongest_neuron]:.3f})")

## ⚡ THE TRIALS OF MASTERY

Master Pai-Torch evaluates your spiritual observation skills through sacred trials.

In [ ]:
# ⚡ THE TRIALS OF MASTERY

def test_your_wisdom(model):
    """Master Pai-Torch's evaluation of your spiritual observation mastery."""
    
    print("🧘 Master Pai-Torch tests your understanding...\n")
    
    # Test 1: Model architecture validation
    total_params = sum(p.numel() for p in model.parameters())
    expected_params = 4*8 + 8 + 8*3 + 3  # input->hidden + hidden_bias + hidden->output + output_bias
    assert total_params == expected_params, f"Expected {expected_params} parameters, got {total_params}"
    print("✓ Architecture Test: Your network structure is sound")
    
    # Test 2: Output shape validation
    test_features = torch.randn(5, 4)  # 5 visitors, 4 features each
    with torch.no_grad():
        raw_outputs = model(test_features)
        probabilities = torch.softmax(raw_outputs, dim=1)
    
    assert raw_outputs.shape == (5, 3), f"Expected output shape (5, 3), got {raw_outputs.shape}"
    print("✓ Output Shape Test: Your predictions have correct dimensions")
    
    # Test 3: Probability validation
    prob_sums = probabilities.sum(dim=1)
    assert torch.allclose(prob_sums, torch.ones(5), atol=1e-6), "Probabilities don't sum to 1!"
    print("✓ Probability Test: Your softmax outputs valid probabilities")
    
    # Test 4: Training effectiveness
    final_accuracy = accuracy_history[-1]
    assert final_accuracy > 0.7, f"Accuracy {final_accuracy:.3f} too low - the spirits are not pleased"
    print(f"✓ Learning Test: Your model achieved {final_accuracy:.3f} accuracy")
    
    # Test 5: Loss convergence
    final_loss = loss_history[-1]
    initial_loss = loss_history[0]
    improvement = (initial_loss - final_loss) / initial_loss
    assert improvement > 0.5, f"Loss improved by only {improvement:.1%} - more training needed"
    print(f"✓ Convergence Test: Loss improved by {improvement:.1%}")
    
    # Test 6: Feature processing
    # Create extreme examples to test if model responds appropriately
    extreme_novice = torch.tensor([[1.2, 0.2, 15.0, 0.3]])  # Fast, bad posture, many questions, shallow breathing
    extreme_master = torch.tensor([[0.2, 1.0, 0.0, 1.0]])    # Slow, perfect posture, no questions, deep breathing
    
    novice_probs = get_class_probabilities(model, extreme_novice)
    master_probs = get_class_probabilities(model, extreme_master)
    
    novice_prediction = torch.argmax(novice_probs, dim=1).item()
    master_prediction = torch.argmax(master_probs, dim=1).item()
    
    # These might not always be perfect due to randomness, but let's check general behavior
    if novice_prediction == 0:  # Correctly identified as novice
        print("✓ Extreme Example Test: Correctly identified extreme novice")
    else:
        print(f"⚠ Extreme Example: Extreme novice classified as {stage_names[novice_prediction]}")
    
    if master_prediction == 2:  # Correctly identified as master
        print("✓ Extreme Example Test: Correctly identified extreme master")
    else:
        print(f"⚠ Extreme Example: Extreme master classified as {stage_names[master_prediction]}")
    
    print("\n🎉 Master Pai-Torch nods with deep approval...")
    print('"Your eyes now see beyond the surface, Grasshopper."')
    print('"You have learned to perceive the invisible patterns of spiritual growth."')
    print('"The temple visitors\'attle mysteries are revealed to your trained perception!"')

# Run the wisdom test
test_your_wisdom(model)

## 🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

Master Pai-Torch presents four paths to deepen your understanding of multi-variable multi-class classification.

### Extension 1: Cook Oh-Pai-Timizer's Batch Feeding Wisdom
*"When cooking for many, efficiency and consistency matter most!"*

Cook Oh-Pai-Timizer bustles over, carrying a large pot of steaming soup. "Ah, Grasshopper! I see you've learned to observe one visitor at a time, like preparing a single bowl of rice. But what happens when the entire monastery arrives for the evening meal? You must learn to process many visitors simultaneously—batch classification for efficient spiritual assessment!"

**NEW CONCEPTS:** Batch processing efficiency, mini-batch training, vectorized operations  
**DIFFICULTY:** +15% (still Dan 1, but with batch optimization)

In [ ]:
# 🍜 EXTENSION 1: BATCH SPIRITUAL ASSESSMENT

def train_with_batches(model, features, labels, batch_size=32, epochs=500, learning_rate=0.01):
    """
    Train the spiritual observer using mini-batches for efficiency.
    """
    # TODO: Implement mini-batch training
    # Hint: Use torch.utils.data.DataLoader or manually create batches
    # Compare training time and final accuracy with full-batch training
    pass

def compare_batch_sizes():
    """
    Compare training efficiency across different batch sizes.
    """
    # TODO: Train models with batch_size=[1, 8, 32, 64, 300] (full batch)
    # Measure training time and final accuracy for each
    # Plot results showing Cook's wisdom about batch efficiency
    pass

# TRIAL: Train your model using different batch sizes
# SUCCESS: Understand the trade-off between training speed and convergence stability
# COOK'S WISDOM: "The right batch size feeds many without burning the soup!"

### Extension 2: He-Ao-World's Measurement Mishaps
*"Oh dear! My old eyes mixed up some of the observation records..."*

He-Ao-World shuffles over, looking apologetic while holding a stack of observation scrolls. "Grasshopper, I must confess... while copying Master Pai-Torch's visitor observations, these old hands made some errors. Some walking speeds got recorded in different units, some posture scores were written upside down, and I might have double-counted some breathing measurements. The data looks rather... inconsistent now."

**NEW CONCEPTS:** Data preprocessing, feature normalization, handling inconsistent scales  
**DIFFICULTY:** +25% (still Dan 1, but messier real-world data)

In [ ]:
# 🧹 EXTENSION 2: CLEANING CORRUPTED OBSERVATIONS

def corrupt_observations(features, corruption_level=0.3):
    """
    Simulate He-Ao-World's data recording mistakes.
    """
    # TODO: Randomly corrupt some features:
    # - Scale some walking speeds by random factors (0.1x to 10x)
    # - Flip some posture scores (1.0 - original_score)
    # - Add random offsets to some question frequencies
    # - Multiply some breathing depths by random values
    pass

def preprocess_features(corrupted_features):
    """
    Clean and normalize the corrupted observation data.
    """
    # TODO: Implement robust preprocessing:
    # - Detect and handle outliers using statistical methods
    # - Apply feature normalization (standardization or min-max scaling)
    # - Store preprocessing parameters for consistent application
    pass

def compare_preprocessing_methods(original_features, corrupted_features, labels):
    """
    Compare model performance with different preprocessing approaches.
    """
    # TODO: Train models on:
    # 1. Original clean data
    # 2. Corrupted data (no preprocessing)
    # 3. Corrupted data with standardization
    # 4. Corrupted data with min-max scaling
    # 5. Corrupted data with robust preprocessing (outlier handling)
    pass

# TRIAL: Handle He-Ao-World's data corruption gracefully
# SUCCESS: Achieve similar accuracy to clean data through smart preprocessing
# HE-AO'S WISDOM: "Sometimes the messiest data teaches the most valuable lessons!"

### Extension 3: Master Ao-Tougrad's Confidence Mysteries
*"True wisdom lies not just in knowing the answer, but in knowing how certain you are..."*

Master Ao-Tougrad materializes from the shadows, speaking in their characteristically enigmatic way. "Your model speaks in probabilities, but do you understand what these numbers truly mean? A prediction of 90% confidence carries different weight than one of 51%. Learn to interpret the uncertainty in your spiritual observations—some visitors reveal their nature clearly, others remain shrouded in mystery."

**NEW CONCEPTS:** Prediction confidence analysis, uncertainty quantification, calibration  
**DIFFICULTY:** +35% (still Dan 1, but deeper interpretation)

In [ ]:
# ⚡ EXTENSION 3: UNDERSTANDING PREDICTION CONFIDENCE

def analyze_prediction_confidence(model, features, labels):
    """
    Analyze how confident the model is in its predictions.
    """
    # TODO: Calculate confidence metrics:
    # - Maximum probability for each prediction
    # - Entropy of probability distributions
    # - Difference between top two probabilities
    # Group predictions by confidence level and analyze accuracy
    pass

def create_confidence_visualization(model, features, labels):
    """
    Visualize the relationship between confidence and accuracy.
    """
    # TODO: Create plots showing:
    # - Confidence distribution for correct vs incorrect predictions
    # - Accuracy vs confidence bins
    # - Feature patterns that lead to high/low confidence
    pass

def implement_rejection_option(model, features, labels, confidence_threshold=0.7):
    """
    Implement a rejection option: refuse to classify when confidence is too low.
    """
    # TODO: 
    # - Calculate prediction confidences
    # - Only make predictions above threshold
    # - Analyze the trade-off between coverage and accuracy
    # - Find optimal threshold that maximizes reliable predictions
    pass

# TRIAL: Understand when your model is uncertain and why
# SUCCESS: Identify the confidence threshold that gives 95%+ accuracy on accepted predictions
# AO-TOUGRAD'S WISDOM: "The wise classifier knows when to say 'I do not know.'"

### Extension 4: Suki's Advanced Behavioral Patterns
*"Meow meow purr." (Translation: "Your features are primitive, human. Observe deeper patterns.")*

Suki sits regally beside Master Pai-Torch, tail swishing with what appears to be mild disapproval. Master Pai-Torch translates her mystical communications: "The sacred cat observes that you process each behavioral feature independently, like counting fish without noticing the pattern of the school. True spiritual insight comes from understanding how behaviors interact—perhaps walking speed and posture combine in ways more subtle than mere addition."

**NEW CONCEPTS:** Feature interactions, polynomial features, feature engineering  
**DIFFICULTY:** +45% (still Dan 1, but advanced feature understanding)

In [ ]:
# 🐱 EXTENSION 4: SUKI'S FEATURE INTERACTION WISDOM

def create_interaction_features(features):
    """
    Create new features based on interactions between original behaviors.
    """
    # TODO: Engineer meaningful feature interactions:
    # - walking_speed * posture_score (coordination index)
    # - question_frequency / breathing_depth (anxiety ratio)
    # - walking_speed / breathing_depth (urgency factor)
    # - posture_score * breathing_depth (mindfulness indicator)
    # Return expanded feature tensor
    pass

def create_polynomial_features(features, degree=2):
    """
    Create polynomial feature combinations up to specified degree.
    """
    # TODO: Generate polynomial features:
    # - Degree 1: original features
    # - Degree 2: all pairwise products, squares
    # - Be careful about feature explosion!
    pass

def analyze_feature_importance(model, original_features, enhanced_features, labels):
    """
    Compare model performance and feature importance with enhanced features.
    """
    # TODO: Train models with:
    # 1. Original 4 features
    # 2. Original + hand-crafted interactions
    # 3. Original + polynomial features
    # 
    # Analyze which new features are most useful
    # Visualize decision boundaries in 2D projections
    pass

def visualize_interaction_effects(features, labels):
    """
    Create visualizations showing how feature interactions reveal spiritual stages.
    """
    # TODO: Create scatter plots of feature pairs colored by class
    # Show how combinations like (walking_speed, posture_score) 
    # separate classes better than individual features
    pass

# TRIAL: Discover which behavioral combinations most clearly reveal spiritual development
# SUCCESS: Achieve higher accuracy using intelligently engineered features
# MASTERY: Understand why certain feature interactions matter for spiritual classification
# SUKI'S WISDOM: "Purr meow purr." ("Behaviors dance together, human. Learn the choreography.")

## 🔥 CORRECTING YOUR FORM: A STANCE IMBALANCE

Master Pai-Torch observes your classification ritual with a keen eye. "Your eager mind grasps the patterns well, Grasshopper, but I detect instability in your training stance. See how your gradient flow wavers?"

A previous disciple left this flawed multi-class training ritual. The form has become unsteady—can you restore proper technique?

In [ ]:
# 🔥 CORRECTING YOUR FORM: A STANCE IMBALANCE

def unstable_classification_training(model, features, labels, epochs=1000):
    """This multi-class training stance has lost its balance - your form needs correction! 🥋"""
    
    # Using the wrong loss function for multi-class classification
    criterion = nn.MSELoss()  # This is for regression, not classification!
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        # Forward pass
        raw_scores = model(features)
        
        # Converting labels to one-hot for MSE (unnecessary and problematic)
        one_hot_labels = torch.zeros(labels.size(0), 3)
        one_hot_labels.scatter_(1, labels.unsqueeze(1), 1)
        
        loss = criterion(raw_scores, one_hot_labels)
        
        # Backward pass - but missing something critical!
        loss.backward()
        optimizer.step()
        # The Gradient Spirits are not being properly dismissed!
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    return model

# DEBUGGING CHALLENGE: This training ritual has THREE critical errors!
# ERROR 1: Wrong loss function - MSE is for regression, not multi-class classification
# ERROR 2: Unnecessary one-hot encoding - CrossEntropyLoss expects class indices
# ERROR 3: Missing optimizer.zero_grad() - gradients accumulate without proper clearing
#
# MASTER'S WISDOM: "The untrained mind applies tools without understanding their purpose.
#                   Each loss function serves its sacred duty—regression seeks closeness,
#                   classification seeks probability. Know the difference, and choose wisely."
#
# HINT: The correct form uses CrossEntropyLoss with raw class indices, 
#       and always clears gradients before each backward pass.

print("🥋 Master Pai-Torch challenges you to identify and correct the three errors in this training ritual.")
print("   Can you restore proper multi-class classification form?")